<a href="https://colab.research.google.com/github/DGkurnia/sentiment/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Nama : Derfansyah Guswiranata Kurnia
- Iddicoding: dgkurnia
- email dicoding: weerakurnia@gmai.com

#Insiasi

In [ ]:
#Perbaikan untuk upgrade colab
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [2]:
#Perpustakaan tahap awal
!pip install selenium beautifulsoup4 pandas

In [3]:
#Impor Bagian awal
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:
#Next Import
import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

In [5]:
#Tambahan
from selenium.webdriver.chrome.service import Service

In [6]:
#Instalasi sastrawi
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [8]:
#Impor Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia
#ke wordcloud

In [9]:
#seksi wordcloud
!pip install wordcloud
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
# ke NITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 6.9 MB/s eta 0:00:00


In [10]:
#NITK
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
#impor file tambahan (google drive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Perpustakaan lain untuk analisis

In [12]:
#Seksi Io dan lain-lain
from io import StringIO
import concurrent.futures #untuk pemrograman paralel
from concurrent.futures import ThreadPoolExecutor

In [13]:
#Interpolasi untuk perbaikan: Grafik
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
#perpustakaan untuk interpolasi
from scipy.interpolate import interp1d

In [14]:
#Bagian lain
import csv
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, f1_score

### Persiapan untuk analisis

In [15]:
#Ekstraksi
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer  #sentimen
#pemisahan
from sklearn.model_selection import train_test_split as ttsplit
#ke klasifikasi

In [16]:
#klasifikasi randomforest
from sklearn.ensemble import RandomForestClassifier as rfclasifi #teknik 'randomfprest'
from sklearn.tree import DecisionTreeClassifier #teknik 'keputusan'
from sklearn.svm import SVC #teknik SVC
from sklearn.linear_model import LogisticRegression

## Pengambilan ulasan dari situs Tokopedia

In [17]:
#Tahap awal pengambilan
urllist = [
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-jazlyn-black-jazlyn-black-s?extParam=ivf%3Dfalse%26keyword%3Dbaju%26search_id%3D20250114050649233F63B3CB036A323GNH%26src%3Dsearch",
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gribson-rayon-navy-s",
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-lavrenti-white-lavrenti-white-s"
]

In [18]:
#Persiapan untuk proses pengambilan list
def review_scrapper(url): #Fungsi ulasan
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  #pelacakan ulasan
  ulasan = []
  for rev in soup.find_all('div', class_='review-class'):
    teks = rev.find('div', class_='review-content').text.strip()
    ulasan.append(teks)
  return ulasan

Pengambilan nilai ulasan per potongan

In [19]:
#fungsi pemotongan analisis
def chunk_and_execute(urls, chunk_size=10):
    totulas = []

    def process_urls(chunk):
        nonlocal totulas

        with ThreadPoolExecutor() as executor:
            results = executor.map(review_scrapper, chunk)

        # Extend the main list with scraped data
        for result in results:
            totulas.extend(result)

    num_chunks = int(len(urls) / chunk_size) + (len(urls) % chunk_size != 0)

    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(urls))

        chunk_url_list = urls[start_idx:end_idx]

        process_urls(chunk_url_list)

        # Penyimpanan hasil sementara
        pd.DataFrame(totulas[:], columns=['Review']).to_csv('intermediate_reviews{}.csv'.format(i), index=False)

    return totulas

#ke tahap akhir

In [ ]:
#penyatuan ke hasil akhir
hslakhir = chunk_and_execute(urllist)
#ke penyetelan data csv

totulas adalah 'total ulasan'

In [ ]:
#persiapan df untuk pembersihan
dfori = pd.DataFrame(hslakhir, columns=['Review'])
dfkopi = dfori.copy() #Jangan pakai data asli
dfkopi.to_csv('tokopedia_reviews.csv', index=False)

In [ ]:
#Inspeksi dataset
dfinfo = dfkopi.info()
print(dfinfo)

# Tahap awal (pembersihan)

In [ ]:
#persiapan untuk pembersihan


In [ ]:
#Persiapan tahap awal
